In [1]:
import psycopg2 as pg
import yaml
from pathlib import Path
import os
import pandas as pd

In [3]:
schema_path = '../misc/schemas.yaml'
with open(schema_path) as schema_file:
    config = yaml.load(schema_file)

In [4]:
config

[{'name': 'complaints_users',
  'size': 'medium',
  'columns': ['complaint_id',
   'complaint_text',
   'was_user_disputed',
   'date',
   'product_id',
   'issue_id'],
  'schema': 'complaint_id integer primary key,\ncomplaint_text text not null,\nwas_user_disputed varchar(3),\ndate timestamp without time zone not null,\nproduct_id integer,\nissue_id integer\n'},
 {'name': 'complaints_companies',
  'size': 'medium',
  'columns': ['complaint_id',
   'company',
   'company_response_to_user',
   'company_response_to_public',
   'was_response_timely',
   'date'],
  'schema': 'complaint_id integer primary key,\ncompany varchar(1000) not null,\ncompany_response_to_user varchar(32),\ncompany_response_to_public varchar(128),\nwas_response_timely varchar(3),\ndate timestamp without time zone not null\n'},
 {'name': 'products',
  'size': 'small',
  'columns': ['product_id', 'main_product', 'sub_product'],
  'schema': 'product_id integer primary key,\nmain_product varchar(80) not null,\nsub_produ

In [5]:
for table in config:
    print(table.get('name'))
    print(table.get('schema'))

complaints_users
complaint_id integer primary key,
complaint_text text not null,
was_user_disputed varchar(3),
date timestamp without time zone not null,
product_id integer,
issue_id integer

complaints_companies
complaint_id integer primary key,
company varchar(1000) not null,
company_response_to_user varchar(32),
company_response_to_public varchar(128),
was_response_timely varchar(3),
date timestamp without time zone not null

products
product_id integer primary key,
main_product varchar(80) not null,
sub_product varchar(44)

issues
issue_id integer primary key,
main_issue varchar(90) not null,
sub_issue varchar(90)



In [11]:
data_path = '../data/'
for table in config:
    table_name = table.get('name')
    table_source = data_path + f"{table_name}.csv"
    table_cols = [str.upper(i) for i in table.get('columns')]
    df = pd.read_csv(table_source)
    df.head(5)
#     df_reorder = df[table_cols]  # rearrange column here
#     df_reorder.to_csv(table_source, index=False)

/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
table = config[0]

table_name = table.get('name')
table_source = data_path + f"{table_name}.csv"
table_cols = [str.upper(i) for i in table.get('columns')]
df = pd.read_csv(table_source)
df.head(5)

,COMPLAINT_ID,COMPLAINT_TEXT,WAS_USER_DISPUTED,DATE,PRODUCT_ID,ISSUE_ID
0,3184195,XXXX and Transunion are reporting incorrectly...,NaN,03/19/2019,26,253
1,3184692,XXXX and Transunion are reporting incorrectly ...,NaN,03/19/2019,26,253
2,3183613,"XXXX, XXXX, and Experian need to remove the co...",NaN,03/18/2019,26,165
3,3182430,"3 company with inconsistencies, violations and...",NaN,03/17/2019,26,253
4,3182218,I have a personal loan from Patriot finance. T...,NaN,03/17/2019,26,165


In [13]:
df[table_cols]

,COMPLAINT_ID,COMPLAINT_TEXT,WAS_USER_DISPUTED,DATE,PRODUCT_ID,ISSUE_ID
0,3184195,XXXX and Transunion are reporting incorrectly...,NaN,03/19/2019,26,253
1,3184692,XXXX and Transunion are reporting incorrectly ...,NaN,03/19/2019,26,253
2,3183613,"XXXX, XXXX, and Experian need to remove the co...",NaN,03/18/2019,26,165
3,3182430,"3 company with inconsistencies, violations and...",NaN,03/17/2019,26,253
4,3182218,I have a personal loan from Patriot finance. T...,NaN,03/17/2019,26,165
5,3182593,Starting in XX/XX/XXXX I started receiving cal...,NaN,03/17/2019,37,18
6,3181958,Card issuer received confirmation of payment i...,NaN,03/16/2019,26,165
7,3181688,YOUR COMPLAINT I am disputing the debt owed to...,NaN,03/16/2019,39,349
8,3182176,I met the requirements for the ability to bene...,NaN,03/16/2019,90,300
9,3181575,"XXXX, XXXX, and Transunion are not taking my c...",NaN,03/16/2019,26,253
